In [1]:
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestClassifier
import catboost as cat
from catboost import CatBoostClassifier
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import log_loss, confusion_matrix, accuracy_score
import xgboost as xgb
import lightgbm as lgb
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [2]:
#importing files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample = pd.read_csv('sample_submission.csv')

In [3]:
def fill_missing_values(data):
    
    '''
    Function to input missing values based on the column object type
    '''
    
    cols = list(data.columns)
    for col in cols:
        if data[col].dtype == 'int64' or data[col].dtype == 'float64':
        
            data[col] = data[col].fillna(data[col].mean())
        
        #elif data[col].dtype == 'O' or data[col].dtype == 'object':
        #    data[col] = data[col].fillna(data[col].mode()[0])
            
        else:
            data[col] = data[col].fillna(data[col].mode()[0])
            
    return data

In [4]:
def freq_encoding(traindata, testdata, *args):
    
    for _ in args:
        train_enc = (traindata.groupby(_).size()) / len(traindata)
        test_enc = (testdata.groupby(_).size()) / len(testdata)
        
        traindata[_] = traindata[_].apply(lambda x: train_enc[x])
        testdata[_] = testdata[_].apply(lambda x: test_enc[x])
    
    return traindata, testdata

In [5]:
def one_hot_encoding(traindata, testdata, *args):
    
    for ii in args:
        traindata = pd.get_dummies(traindata, prefix=[ii], columns=[ii])
        testdata = pd.get_dummies(testdata, prefix=[ii], columns=[ii])
        
    return traindata, testdata

In [6]:
def drop_columns(traindata, testdata, target, *args):
    
    labels = np.array(traindata[target])
    
    columns = []
    for _ in args:
        columns.append(_)
        
    traindata = traindata.drop(columns, axis=1)
    traindata = traindata.drop(target, axis=1)
    testdata = testdata.drop(columns, axis=1)
        
    return labels, traindata, testdata

In [7]:
cols = ['ZONE1', 'ZONE2', 'TIGO', 'user_id', 'MRG']

labels, train1, test1 = drop_columns(train, test, 'CHURN', *cols)

In [9]:
train1 = train1.fillna(-999)
test1 = test1.fillna(-999)

In [43]:
train1 = fill_missing_values(train1)
test1 = fill_missing_values(test1)

In [8]:
one_hot_cols = ['REGION', 'TENURE']

train1, test1 = one_hot_encoding(train1, test1, *one_hot_cols)

In [10]:
freq_col = ['TOP_PACK']

train1, test1 = freq_encoding(train1, test1, *freq_col)

In [11]:
features = np.array(train1)
test_enc = np.array(test1)

#scaler = preprocessing.StandardScaler().fit(features)
#features = scaler.transform(features)
#test_enc = scaler.transform(test_enc)

In [12]:
#using StandardScaler function to scale the numeric features 

scaler = preprocessing.StandardScaler().fit(features[:, :])
features[:, :] = scaler.transform(features[:, :])
features[:, :]

array([[ 2.11347837,  0.77723114,  2.22374343, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       [ 0.16341176,  0.77101117,  0.16829127, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       [-0.26652419,  0.7171048 , -0.27496434, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       ...,
       [-0.38936303,  0.71503147, -0.39626525, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       [-0.65024202, -1.36036399, -0.50075666, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       [-0.65024202, -1.36036399, -0.65340499, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293]])

In [13]:
#using StandardScaler function to scale the numeric features 

scaler1 = preprocessing.StandardScaler().fit(test_enc[:, :])
test_enc[:, :] = scaler1.transform(test_enc[:, :])
test_enc[:, :]

array([[-0.65111099, -1.36390268, -0.65551124, ..., -0.14692214,
        -0.07618062,  0.23361125],
       [-0.65111099, -1.36390268, -0.50292553, ..., -0.14692214,
        -0.07618062,  0.23361125],
       [-0.20772843,  0.73999703, -0.15677221, ..., -0.14692214,
        -0.07618062,  0.23361125],
       ...,
       [-0.22302276,  0.72339822, -0.23238455, ...,  6.80632622,
        -0.07618062, -4.28061578],
       [-0.11596247,  0.71717366, -0.12410768, ..., -0.14692214,
        -0.07618062,  0.23361125],
       [-0.65111099, -1.36390268, -0.65551124, ..., -0.14692214,
        -0.07618062,  0.23361125]])

In [14]:
## Randomly splitting into train and test data
nr.seed(9988)
indx = range(features.shape[0])
indx = ms.train_test_split(indx, test_size = 0.3)
x_train = features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
x_test = features[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [15]:
'''
    Randomly splitting the test data into two parts; one for validation during training
    and the other for testing the model
'''

np.random.seed(9988)
indx1 = range(x_test.shape[0])
indx1 = ms.train_test_split(indx1, test_size = 0.5)
x_test1 = x_test[indx1[0],:]
y_test1 = np.ravel(y_test[indx1[0]])
x_test2 = x_test[indx1[1],:]
y_test2 = np.ravel(y_test[indx1[1]])

In [16]:
rf = RandomForestClassifier(n_estimators=400, 
                            max_depth=12,
                            max_features="auto",
                            min_samples_split=2,
                            min_samples_leaf=1,
                            verbose=2)

In [17]:
#preparing the predictions make into the sample file for submission

preds1 = xg.predict_proba(test_enc)
sample1 = pd.read_csv('sample_submission.csv')
test = pd.read_csv('Test.csv')
sample1.user_id = test.user_id
sample1.CHURN = preds1[:, 1]
sample1.to_csv('XGGGGGG.csv', index = False)

In [16]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=10).fit(x_train, y_train)

pred = clf.predict(x_test)
print(f'Log loss: {log_loss(pred, y_test)}')
print(f'Accuracy score: {accuracy_score(pred, y_test)}')

Log loss: 4.325444897654616
Accuracy score: 0.8747666666666667


In [15]:
cols1 = list(train1.columns)
cols2 = list(test1.columns)

In [19]:
for i in cols1:
    if i not in cols2:
        print("No")

In [29]:
t1 = train.dropna()

In [33]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_test1.shape, y_test1.shape)
print(x_test2.shape, y_test2.shape)

(280000, 33) (280000,)
(120000, 33) (120000,)
(60000, 33) (60000,)
(60000, 33) (60000,)


In [16]:
xg = xgb.XGBClassifier(objective='reg:logistic',
                       boosting_type='dart', 
                       learning_rate=0.01, 
                       n_estimators=5000, 
                       max_depth=10,
                       subsample=0.8,
                       base_score=0.7,
                       max_features='auto'
                     )

eval_set = [(x_train, y_train), (x_test1, y_test1)]
xg.fit(x_train, y_train, eval_metric="logloss", eval_set=eval_set, verbose=2, early_stopping_rounds = 70)

[02:27:39] WARNING: /workspace/src/learner.cc:480: 
Parameters: { boosting_type, max_features } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-logloss:1.02949	validation_1-logloss:1.03033
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 70 rounds.
[2]	validation_0-logloss:1.00016	validation_1-logloss:1.00065
[4]	validation_0-logloss:0.97245	validation_1-logloss:0.97259
[6]	validation_0-logloss:0.94618	validation_1-logloss:0.94633
[8]	validation_0-logloss:0.92109	validation_1-logloss:0.92142
[10]	validation_0-logloss:0.89785	validation_1-logloss:0.89789
[12]	validation_0-logloss:0.87544	validation_1-logloss:0.87562
[14]	validation_0-logloss:0.85431	validation

[242]	validation_0-logloss:0.28765	validation_1-logloss:0.28866
[244]	validation_0-logloss:0.28685	validation_1-logloss:0.28790
[246]	validation_0-logloss:0.28607	validation_1-logloss:0.28714
[248]	validation_0-logloss:0.28532	validation_1-logloss:0.28641
[250]	validation_0-logloss:0.28459	validation_1-logloss:0.28570
[252]	validation_0-logloss:0.28391	validation_1-logloss:0.28500
[254]	validation_0-logloss:0.28314	validation_1-logloss:0.28433
[256]	validation_0-logloss:0.28246	validation_1-logloss:0.28366
[258]	validation_0-logloss:0.28178	validation_1-logloss:0.28301
[260]	validation_0-logloss:0.28111	validation_1-logloss:0.28238
[262]	validation_0-logloss:0.28049	validation_1-logloss:0.28176
[264]	validation_0-logloss:0.27984	validation_1-logloss:0.28115
[266]	validation_0-logloss:0.27924	validation_1-logloss:0.28056
[268]	validation_0-logloss:0.27862	validation_1-logloss:0.27997
[270]	validation_0-logloss:0.27806	validation_1-logloss:0.27940
[272]	validation_0-logloss:0.27745	valid

[500]	validation_0-logloss:0.25040	validation_1-logloss:0.25446
[502]	validation_0-logloss:0.25033	validation_1-logloss:0.25440
[504]	validation_0-logloss:0.25025	validation_1-logloss:0.25434
[506]	validation_0-logloss:0.25018	validation_1-logloss:0.25429
[508]	validation_0-logloss:0.25011	validation_1-logloss:0.25423
[510]	validation_0-logloss:0.25002	validation_1-logloss:0.25417
[512]	validation_0-logloss:0.24995	validation_1-logloss:0.25414
[514]	validation_0-logloss:0.24988	validation_1-logloss:0.25409
[516]	validation_0-logloss:0.24981	validation_1-logloss:0.25403
[518]	validation_0-logloss:0.24972	validation_1-logloss:0.25399
[520]	validation_0-logloss:0.24962	validation_1-logloss:0.25394
[522]	validation_0-logloss:0.24955	validation_1-logloss:0.25389
[524]	validation_0-logloss:0.24948	validation_1-logloss:0.25385
[526]	validation_0-logloss:0.24944	validation_1-logloss:0.25379
[528]	validation_0-logloss:0.24937	validation_1-logloss:0.25375
[530]	validation_0-logloss:0.24930	valid

[758]	validation_0-logloss:0.24457	validation_1-logloss:0.25146
[760]	validation_0-logloss:0.24453	validation_1-logloss:0.25146
[762]	validation_0-logloss:0.24450	validation_1-logloss:0.25144
[764]	validation_0-logloss:0.24447	validation_1-logloss:0.25144
[766]	validation_0-logloss:0.24445	validation_1-logloss:0.25143
[768]	validation_0-logloss:0.24442	validation_1-logloss:0.25143
[770]	validation_0-logloss:0.24440	validation_1-logloss:0.25142
[772]	validation_0-logloss:0.24436	validation_1-logloss:0.25141
[774]	validation_0-logloss:0.24434	validation_1-logloss:0.25141
[776]	validation_0-logloss:0.24431	validation_1-logloss:0.25140
[778]	validation_0-logloss:0.24424	validation_1-logloss:0.25139
[780]	validation_0-logloss:0.24420	validation_1-logloss:0.25138
[782]	validation_0-logloss:0.24416	validation_1-logloss:0.25138
[784]	validation_0-logloss:0.24412	validation_1-logloss:0.25138
[786]	validation_0-logloss:0.24409	validation_1-logloss:0.25137
[788]	validation_0-logloss:0.24405	valid

[1014]	validation_0-logloss:0.24073	validation_1-logloss:0.25113
[1016]	validation_0-logloss:0.24071	validation_1-logloss:0.25113
[1018]	validation_0-logloss:0.24066	validation_1-logloss:0.25113
[1020]	validation_0-logloss:0.24060	validation_1-logloss:0.25114
[1022]	validation_0-logloss:0.24057	validation_1-logloss:0.25114
[1024]	validation_0-logloss:0.24054	validation_1-logloss:0.25114
[1026]	validation_0-logloss:0.24049	validation_1-logloss:0.25114
[1028]	validation_0-logloss:0.24047	validation_1-logloss:0.25114
[1030]	validation_0-logloss:0.24044	validation_1-logloss:0.25115
[1032]	validation_0-logloss:0.24040	validation_1-logloss:0.25116
[1034]	validation_0-logloss:0.24037	validation_1-logloss:0.25116
[1036]	validation_0-logloss:0.24036	validation_1-logloss:0.25116
[1038]	validation_0-logloss:0.24033	validation_1-logloss:0.25116
[1040]	validation_0-logloss:0.24030	validation_1-logloss:0.25116
[1042]	validation_0-logloss:0.24028	validation_1-logloss:0.25116
[1044]	validation_0-loglo

XGBClassifier(base_score=0.7, booster='gbtree', boosting_type='dart',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=10, max_features='auto', min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=5000,
              n_jobs=0, num_parallel_tree=1, objective='reg:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [26]:
kf = KFold(n_splits=5,random_state=2020,shuffle=True)

In [24]:
scores = []
test_scores = np.zeros(len(test))
for fold,(tr_in,te_in) in enumerate(kf.split(train1)):
    print(f"================================Fold{fold}====================================")
    X_train,X_test = train1.iloc[tr_in],train1.iloc[te_in]
    y_train,y_test = train['CHURN'].iloc[tr_in],train['CHURN'].iloc[te_in]
    xg.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test,y_test)],early_stopping_rounds=200,use_best_model=True)
    scores.append(log_loss(y_test,xg.predict(X_test)))
    test_scores += cat.predict(test_enc)
    

================================Fold0====================================
[22:15:30] WARNING: /workspace/src/learner.cc:480: 
Parameters: { boosting_type, max_features } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:0.63920	validation_1-rmse:0.63929
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 200 rounds.
[1]	validation_0-rmse:0.63409	validation_1-rmse:0.63420
[2]	validation_0-rmse:0.62892	validation_1-rmse:0.62916
[3]	validation_0-rmse:0.62416	validation_1-rmse:0.62419
[4]	validation_0-rmse:0.61920	validation_1-rmse:0.61929
[5]	validation_0-rmse:0.61430	validation_1-rmse:0.61445
[6]	validation_0-rmse:0.60954	validation_1-rmse:0.60967
[7]	validation_0-rm

KeyboardInterrupt: 